In [ ]:
import pandas as pd
import numpy as np
import pickle

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    try:
        with open(name + '.pkl', 'rb') as f:
            return pickle.load(f)    
    except FileNotFoundError as e:
        return False;


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# This notebook is for vectorizing observations. It is separate due to lack of RAM and because of different logic that is being applied while transforming data. All other data is vectorized in the "vectorizing" notebook.

## The idea is to create additional colums from raw data by dividing original values into 1 < number < 5 of groups. 
## Example: instead of keeping original values for persons weight and than averaging it (or similar) we will replace column "Body weight" with columns "Body_weight<20", "Body_weight>20<50" etc. Then we will go through data and
## add number one multiplied by some penalty to the group where the original data would belong. All observations are than aggregated for each patient.
## Penalty is a factor that gives more importance to the recent observations than to the old data.   

In [ ]:
observations = pd.read_csv('/content/drive/MyDrive/SIAP/data/observations.csv')
patients = pd.read_csv('/content/drive/MyDrive/SIAP/data/patients.csv')
patients_death = patients[['Id', 'DEATHDATE']]

obtype_borders = load_obj("/content/drive/MyDrive/SIAP/helpers/obtype_borders")
print(obtype_borders)

# Use only numeric data as it holds the most information and is of the greatest importance. 
observations = observations.loc[observations['TYPE'] == 'numeric']

observations['YEARS_FROM_DEATH_WEIGHT'] = np.zeros(observations.shape[0], dtype=float)
observations['BUCKET_NAME'] = np.empty(observations.shape[0], dtype=str)

observations = observations.join(patients_death.set_index('Id'), on='PATIENT' , how='left', lsuffix='L')

observations['DEATHDATE'] = pd.to_datetime(observations['DEATHDATE']).dt.date
observations['DATE'] = pd.to_datetime(observations['DATE']).dt.date

observations['yfd'] = (observations['DEATHDATE'] - observations['DATE']).dt.days / 365


{'Body Height': [45.1, 86.3, 147.1, 163.8, 173.5, 198.7], 'Pain severity - 0-10 verbal numeric rating [Score] - Reported': [0.0, 1.0, 2.0, 3.0, 3.0, 10.0], 'Body Weight': [1.8, 13.7, 51.3, 74.5, 85.7, 168.7], 'Body Mass Index': [11.5, 17.8, 24.0, 27.8, 29.6, 51.8], 'Diastolic Blood Pressure': [40.0, 75.0, 79.0, 82.0, 86.0, 123.0], 'Systolic Blood Pressure': [40.0, 111.0, 118.0, 124.0, 132.0, 202.0], 'Heart rate': [50.0, 68.0, 77.0, 85.0, 94.0, 200.0], 'Respiratory rate': [12.0, 13.0, 14.0, 15.0, 15.0, 40.0], 'Total Cholesterol': [150.0, 169.6, 180.3, 191.2, 205.0, 304.9], 'Triglycerides': [100.0, 113.8, 127.5, 141.2, 164.0, 516.7], 'Low Density Lipoprotein Cholesterol': [50.6, 77.5, 88.7, 100.3, 126.0, 199.9], 'High Density Lipoprotein Cholesterol': [14.0, 52.5, 62.4, 68.3, 74.2, 80.0], 'Oxygen Saturation': [78.0, 92.0, 95.0, 98.0], 'Lactate [Mass/volume] in Blood': [0.0, 1.7, 2.9, 4.0], 'Mean blood pressure': [20.1, 58.2, 84.5, 100.0], 'Body temperature': [36.1, 37.5, 38.0, 39.1, 40.6

In [ ]:
 observation_splits = np.array_split(observations, 10)

In [ ]:
morethan30 = 0.2
morethan20 = 0.4
morethan5 = 0.6
morethan1 = 0.9
lessthan1 = 1

ignore_less_than_0_3 = 0.0

def transform_row_observations(r):
    observation = r['DESCRIPTION']
    value = float(r['VALUE'])
    if observation in obtype_borders:
      borders = obtype_borders[observation]
      for i in range(1, len(borders)):
        if value <= borders[i]:
          r['BUCKET_NAME'] = 'o_' + observation + "_" + str(i-1)
          break

    yfd = r['yfd']
    if yfd >= 30:
        r['YEARS_FROM_DEATH_WEIGHT'] = morethan30
    elif yfd < 30 and yfd >= 20:
        r['YEARS_FROM_DEATH_WEIGHT'] = morethan20
    elif yfd < 20 and yfd >= 5:
        r['YEARS_FROM_DEATH_WEIGHT'] = morethan5
    elif yfd < 5 and yfd >= 1:
        r['YEARS_FROM_DEATH_WEIGHT'] = morethan1
    elif yfd < 1 and yfd >= 0.3:
      r['YEARS_FROM_DEATH_WEIGHT'] = lessthan1
    else:
      r['YEARS_FROM_DEATH_WEIGHT'] = ignore_less_than_0_3
    return r

for index, split in enumerate(observation_splits):
  observations_with_time_weight = split.apply(transform_row_observations, axis=1)
  observations_with_time_weight['BUCKET_NAME'].replace('', np.nan, inplace=True)
  observations_with_time_weight = observations_with_time_weight[observations_with_time_weight['BUCKET_NAME'].notna()]

  observations_pivoted = pd.pivot_table(observations_with_time_weight, index=['PATIENT'], columns=['BUCKET_NAME'], values=['YEARS_FROM_DEATH_WEIGHT'], aggfunc="sum", fill_value=0)
  normalized_observations_pivoted=(observations_pivoted-observations_pivoted.min())/(observations_pivoted.max()-observations_pivoted.min())
  normalized_observations_pivoted.columns = normalized_observations_pivoted.columns.droplevel(0)

  normalized_observations_pivoted.to_csv('/content/drive/MyDrive/SIAP/data/prepared/observations_part_' + str(index) + '.csv', encoding='utf-8', index=True)



In [ ]:
combined = pd.read_csv('/content/drive/MyDrive/SIAP/data/prepared/observations_part_' + str(0) + '.csv')

for i in range(1, 10):
  part = pd.read_csv('/content/drive/MyDrive/SIAP/data/prepared/observations_part_' + str(i) + '.csv')
  print(part.shape)
  combined = pd.concat([combined, part], axis=0, ignore_index=False)


(2203, 596)
(2173, 590)
(2114, 591)
(2101, 591)
(2043, 596)
(2111, 596)
(2108, 588)
(2047, 596)
(2062, 589)


In [ ]:
print(len(combined['PATIENT'].unique()))

20981


In [ ]:
combined = combined.groupby(combined['PATIENT']).aggregate('sum')
combined = combined.reset_index()
combined.head()


,PATIENT,o_Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma_0,o_Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma_1,o_Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma_2,o_Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma_3,o_Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma_4,o_Albumin [Mass/volume] in Serum or Plasma_0,o_Albumin [Mass/volume] in Serum or Plasma_1,o_Albumin [Mass/volume] in Serum or Plasma_2,o_Albumin [Mass/volume] in Serum or Plasma_3,o_Albumin [Mass/volume] in Serum or Plasma_4,o_Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma_0,o_Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma_1,o_Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma_2,o_Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma_3,o_Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma_4,o_American house dust mite IgE Ab in Serum_0,o_American house dust mite IgE Ab in Serum_1,o_American house dust mite IgE Ab in Serum_2,o_Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma_0,o_Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma_1,o_Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma_2,o_Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma_3,o_Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma_4,o_Basophils [#/volume] in Blood by Automated count_0,o_Basophils [#/volume] in Blood by Automated count_1,o_Basophils [#/volume] in Blood by Automated count_3,o_Basophils/100 leukocytes in Blood by Automated count_0,o_Basophils/100 leukocytes in Blood by Automated count_1,o_Basophils/100 leukocytes in Blood by Automated count_2,o_Basophils/100 leukocytes in Blood by Automated count_3,o_Basophils/100 leukocytes in Blood by Automated count_4,o_Bicarbonate [Moles/volume] in Arterial blood_0,o_Bicarbonate [Moles/volume] in Arterial blood_1,o_Bicarbonate [Moles/volume] in Arterial blood_2,o_Bicarbonate [Moles/volume] in Arterial blood_3,o_Bicarbonate [Moles/volume] in Arterial blood_4,o_Bilirubin.total [Mass/volume] in Serum or Plasma_0,o_Bilirubin.total [Mass/volume] in Serum or Plasma_1,o_Bilirubin.total [Mass/volume] in Serum or Plasma_2,...,o_Weight difference [Mass difference] --pre dialysis - post dialysis_0,o_Weight difference [Mass difference] --pre dialysis - post dialysis_1,o_Weight difference [Mass difference] --pre dialysis - post dialysis_2,o_Weight difference [Mass difference] --pre dialysis - post dialysis_3,o_Weight difference [Mass difference] --pre dialysis - post dialysis_4,o_Weight-for-length Per age and sex_0,o_Weight-for-length Per age and sex_1,o_Weight-for-length Per age and sex_2,o_Weight-for-length Per age and sex_3,o_Weight-for-length Per age and sex_4,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_0,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_1,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_2,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_3,o_What number best describes how during the past week pain has interfered with your enjoyment of life?_4,o_What number best describes how during the past week pain has interfered with your general activity?_0,o_What number best describes how during the past week pain has interfered with your general activity?_1,o_What number best describes how during the past week pain has interfered with your general activity?_2,o_What number best describes how during the past week pain has interfered with your general activity?_3,o_What number best describes how during the past week pain has interfered with your general activity?_4,o_What number best describes your pain on average in the pa

In [ ]:
combined.to_csv('/content/drive/MyDrive/SIAP/data/prepared/observations_combined_with_buckets.csv', encoding='utf-8', index=False)
